In [ ]:
import os
import numpy as np
import pandas as pd
import gc

In [ ]:
!ls ../input/hm-public-submissions

# **Adding LSTM models to the ensemble**

# Note
* I am not sure whether I should get any credit for this notebook as this is based(mostly) on the work of others.
* I added two more submissions to the ensemble and tried a few options with them.
* This notebooks achieves a score of 0.0234

# To ensemble I used submissions from 7 public notebooks:
* LB: 0.0217 - https://www.kaggle.com/tarique7/hnm-exponential-decay-with-alternate-items/notebook
* LB: 0.0220 - https://www.kaggle.com/code/hengzheng/time-is-our-best-friend-v2/notebook
* LB: 0.0221 - https://www.kaggle.com/astrung/lstm-sequential-modelwith-item-features-tutorial
* LB: 0.0223 - https://www.kaggle.com/code/astrung/lstm-model-with-item-infor-fix-missing-last-item/notebook
* LB: 0.0225 - https://www.kaggle.com/lunapandachan/h-m-trending-products-weekly-add-test/notebook
* LB: 0.0226 - https://www.kaggle.com/lichtlab/0-0226-byfone-chris-combination-approach/data?scriptVersionId=89289696
* LB: 0.0227 - https://www.kaggle.com/code/hechtjp/h-m-eda-rule-base-by-customer-age

In [ ]:
sub2 = pd.read_csv('../input/handmbestperforming/hnm-exponential-decay-with-alternate-items.csv').sort_values('customer_id').reset_index(drop=True)
sub5 = pd.read_csv('../input/handmbestperforming/time-is-our-best-friend-v2.csv').sort_values('customer_id').reset_index(drop=True)
sub3 = pd.read_csv('../input/handmbestperforming/lstm-sequential-modelwith-item-features-tutorial.csv').sort_values('customer_id').reset_index(drop=True)
sub4 = pd.read_csv('../input/handmbestperforming/lstm-model-with-item-infor-fix-missing-last-item.csv').sort_values('customer_id').reset_index(drop=True)

sub0 = pd.read_csv('../input/handmbestperforming/0-0226-byfone-chris-combination-approach.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('../input/handmbestperforming/h-m-trending-products-weekly-add-test.csv').sort_values('customer_id').reset_index(drop=True)
sub6 = pd.read_csv('../input/handmbestperforming/rule-based-by-customer-age.csv').sort_values('customer_id').reset_index(drop=True)

In [ ]:
# How many predictions are in common between models

print((sub0['prediction']==sub0['prediction']).mean())
print((sub0['prediction']==sub1['prediction']).mean())
print((sub0['prediction']==sub2['prediction']).mean())
print((sub0['prediction']==sub3['prediction']).mean())
print((sub0['prediction']==sub4['prediction']).mean())
print((sub0['prediction']==sub5['prediction']).mean())
print((sub0['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub1['prediction']==sub0['prediction']).mean())
print((sub1['prediction']==sub1['prediction']).mean())
print((sub1['prediction']==sub2['prediction']).mean())
print((sub1['prediction']==sub3['prediction']).mean())
print((sub1['prediction']==sub4['prediction']).mean())
print((sub1['prediction']==sub5['prediction']).mean())
print((sub1['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub2['prediction']==sub0['prediction']).mean())
print((sub2['prediction']==sub1['prediction']).mean())
print((sub2['prediction']==sub2['prediction']).mean())
print((sub2['prediction']==sub3['prediction']).mean())
print((sub2['prediction']==sub4['prediction']).mean())
print((sub2['prediction']==sub5['prediction']).mean())
print((sub2['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub3['prediction']==sub0['prediction']).mean())
print((sub3['prediction']==sub1['prediction']).mean())
print((sub3['prediction']==sub2['prediction']).mean())
print((sub3['prediction']==sub3['prediction']).mean())
print((sub3['prediction']==sub4['prediction']).mean())
print((sub3['prediction']==sub5['prediction']).mean())
print((sub3['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub4['prediction']==sub0['prediction']).mean())
print((sub4['prediction']==sub1['prediction']).mean())
print((sub4['prediction']==sub2['prediction']).mean())
print((sub4['prediction']==sub3['prediction']).mean())
print((sub4['prediction']==sub5['prediction']).mean())
print((sub4['prediction']==sub4['prediction']).mean())
print((sub4['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub5['prediction']==sub0['prediction']).mean())
print((sub5['prediction']==sub1['prediction']).mean())
print((sub5['prediction']==sub2['prediction']).mean())
print((sub5['prediction']==sub3['prediction']).mean())
print((sub5['prediction']==sub4['prediction']).mean())
print((sub5['prediction']==sub6['prediction']).mean())

In [ ]:
print((sub6['prediction']==sub0['prediction']).mean())
print((sub6['prediction']==sub1['prediction']).mean())
print((sub6['prediction']==sub2['prediction']).mean())
print((sub6['prediction']==sub3['prediction']).mean())
print((sub6['prediction']==sub4['prediction']).mean())
print((sub6['prediction']==sub5['prediction']).mean())

In [ ]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
sub0['prediction3'] = sub3['prediction']
sub0['prediction4'] = sub4['prediction']
sub0['prediction5'] = sub5['prediction']
sub0['prediction6'] = sub6['prediction']

del sub1, sub2, sub3, sub4, sub5, sub6
gc.collect()
sub0.head()

In [ ]:
def cust_blend(dt, W = [1,1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]

    #Create a list of all model predictions
    REC = []

    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    REC.append(dt['prediction4'].split())
    REC.append(dt['prediction5'].split())
    REC.append(dt['prediction6'].split())

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])

# sub0['prediction'] = sub0.apply(cust_blend, W = [0.95,1.00,0.55,0.75,0.85,0.65,1.05], axis=1)

sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95,0.85,0.75,0.65,0.55], axis=1)
sub0.head()

In [ ]:
# How many predictions are in common with ensemble

print((sub0['prediction']==sub0['prediction0']).mean())
print((sub0['prediction']==sub0['prediction1']).mean())
print((sub0['prediction']==sub0['prediction2']).mean())
print((sub0['prediction']==sub0['prediction3']).mean())
print((sub0['prediction']==sub0['prediction4']).mean())
print((sub0['prediction']==sub0['prediction5']).mean())
print((sub0['prediction']==sub0['prediction6']).mean())

**The results of comparison seem interesting**

# Make a submission

In [ ]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
del sub0['prediction3']
del sub0['prediction4']
del sub0['prediction5']
del sub0['prediction6']
gc.collect()


sub0.to_csv('submission.csv', index=False)